In [6]:
Packages <- c("dplyr",  "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer", "viridis")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
#source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_grid_search.R")
source("~/oceanography/scripts/LL_biophys.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R")
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained
source("~/oceanography/scripts/format_genetic_parentage_matrix.R")
source("~/oceanography/scripts/format_genetic_kernel_parentage_matrix.R")
source("~/oceanography/scripts/format_biophys_normalized_matrix.R")
source("~/oceanography/scripts/format_biophys_parentage_matrix.R")
set.seed(7)
#source("~/oceanography/scripts/PredictedProportions.R")

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
kernel2012_14 <- fread(file="~/oceanography/empirical_data/genetics/GenKernelsForROMSComp2012-14.csv")

#read in the necessary input data for likelihood functions
load(file= "~/oceanography/script_output/SurveyData/for_likelihood_functions/2022-01-20_AddDestTables.Rdata")
load(file= "~/oceanography/script_output/ROMSDataTables/2021-11-04_SeededParticleInfo.Rdata")
load(file= "~/oceanography/script_output/SurveyData/for_likelihood_functions/2021-11-04_InputSurveyData.Rdata")



In [2]:
AllRecruitsGen <- fread("~/parentage/kernel_fitting/1340_loci/final_results/tables/AllFishObsWithPar.csv")
#head(AllRecruitsGen)
MonsoonRecSampPar <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/RecruitsByMonsoon2012-14ForROMSComp.csv")
#head(MonsoonRecSampPar)

#make one data table to rule them all....

AllRecruitsGenWithMonsoon <- left_join(AllRecruitsGen, MonsoonRecSampPar[, .(fish_indiv, year, monsoon=season)], by=c("fish_indiv", "year"))[
    offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"][year %in% c(2012, 2013, 2014)]

genetic_parentage_matrices <- suppressMessages(format_genetic_parentage_matrix(AllRecruitsGenWithMonsoon, SurveyData))
#fwrite(genetic_parentage_matrices$GenMat2012_4, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2012-14ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2012, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2012ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2013, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2013ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2014, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2014ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMatNEM, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrixNEM2012-14ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMatSWM, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrixSWM2012-14ForROMSComp.csv")


genetic_kernel_matrices <- suppressMessages(format_genetic_kernel_parentage_matrix(AllRecruitsGenWithMonsoon, SurveyData))

#fwrite(genetic_kernel_matrices$KernelGenMat2012_4, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012-14ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2012, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2013, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2013ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2014, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2014ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMatNEM, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixNEM2012-14ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMatSWM, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixSWM2012-14ForROMSComp.csv")


__read in the biophysical data and format for likelihood functions__

In [19]:
SimConn <- fread(file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTableCompleteMetaLongForm08DayPLD.csv")[year %in% c(2012, 2013, 2014) & dest != "CAI"][#filter out CAI as a destination for now, not very well spatially defined
   ,`:=`(daily_particles_rec=as.numeric(daily_particles_rec),daily_particles_released=as.numeric(daily_particles_released))][
    source == "Other" | source == "CAI", source:="unknown"][ #aggregate the CAI and other sites as "unknown for likelihood functions"
    , daily_particles_rec := sum(daily_particles_rec), by=c("date", "source", "dest")][
   , daily_particles_released :=sum(as.numeric(daily_particles_released)), by=c("date", "source")]
total_release_days

[1] 687

In [8]:

biophys_norm_matrices <- format_biophys_normalized_matrix(SimConn=SimConn, total_release_days=687, AddDestSim=AddDestSim)
#fwrite(biophys_norm_matrices$FullBiophysMatNorm, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2012-14ForROMSComp08DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2012, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2012ForROMSComp08DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2013, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2013ForROMSComp08DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2014, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2014ForROMSComp08DayPLD.csv")
#fwrite(biophys_norm_matrices$MonsoonBiophysMatNormNEM, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrixNEMForROMSComp08DayPLD.csv" )
#fwrite(biophys_norm_matrices$MonsoonBiophysMatNormSWM, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrixSWMForROMSComp08DayPLD.csv" )

biophys_parentage_matrices <- format_biophys_parentage_matrix(SimConn=SimConn, total_release_days=687, AddDestSim=AddDestSim)
#fwrite(biophys_parentage_matrices$FullBiophysMat, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2012-14ForROMSComp08DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2012, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2012ForROMSComp08DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2013, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2013ForROMSComp08DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2014, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2014ForROMSComp08DayPLD.csv")
#fwrite(biophys_parentage_matrices$MonsoonBiophysMatNEM, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrixNEMForROMSComp08DayPLD.csv")
#fwrite(biophys_parentage_matrices$MonsoonBiophysMatSWM, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrixSWMForROMSComp08DayPLD.csv")

In [10]:
sum(biophys_parentage_matrices$AnnualBiophysMat2012)+sum(biophys_parentage_matrices$AnnualBiophysMat2013)+sum(biophys_parentage_matrices$AnnualBiophysMat2014)==sum(biophys_parentage_matrices$FullBiophysMat)
#should be true

[1] TRUE

In [44]:
nrow(SimConn)

[1] 955263

In [20]:
unique(SimConn, by="year")

particle_id,source,dest,year,monsoon,date,num_annual_parentage_matches,num_release_days_seasonal,num_release_days_annual,dest_num_females,...,source_num_rec_sampled_annual,source_prop_anem_samp,sim_month,sim_day,sim_year,daily_particles_rec,daily_particles_released,dist_km,bearing,direction
P54642131,unknown,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,4761,4254850688,NA,NA,NA
P113500613,unknown,Palanas,2013,SWM,2012-10-01,21,31,243,112,...,NA,NA,10,10,2012,16641,5779580480,NA,NA,NA
P172397748,unknown,Palanas,2014,SWM,2013-10-01,13,31,200,45,...,NA,NA,10,10,2013,29584,26698513408,NA,NA,NA


In [ ]:
#hmm so the simulation is all representing assigned parentage matches, need to change so that there's only a certain number of parentage matches
SimConnInt2SubSample <- SimConn[!is.na(dest_num_rec_sampled_annual), .SD[sample(.N, dest_num_rec_sampled_annual, replace=TRUE)], by=c("year", "dest")] 


In [32]:

SimConnInt2SubSample <- SimConn[!is.na(dest_num_rec_sampled_annual), .SD[sample(.N, dest_num_rec_sampled_annual, replace=TRUE)], by=c("year", "dest")] #subsample the particle data according to the number of recruits we sampled at each site
SimConnInt2SubSamplePar <- SimConnInt2SubSample[!is.na(dest_num_rec_sampled_annual)& source !="unknown", .SD[sample(.N, num_annual_parentage_matches, replace=TRUE, prob=source_prop_anem_samp)], by="year"] #randomly assign parentage based on the number of matches from that year and use the source prop_samp as the probability weighting
SimConnInt2SubSample[particle_id %!in% SimConnInt2SubSamplePar[, particle_id], source := "unknown"]#and then for every particle that we didn't assign as a match, label the source unknown so it goes in the unassigned row
#nrow(SimConnInt2SubSample[source=="unknown"])+nrow(SimConnInt2SubSample[source!="unknown"]) #should equal 394

[1] 394

[1] 394

In [ ]:
sum(SimConnInt2SubSample[, .N, by=c("year", "dest")][,N])
#hmm for some reason the subsampled assignment matrices are not the right number

# Get the best-fit kernels and profile likelihood confidence intervals in fit_data.ipynb


In [ ]:
sampled_reefs_vec <- as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]) #***I think these index values need to match dimensions of parentage matrix- filter out sand flats and other unsampled sites***
pop_size_vec <- as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]) #vector of pop sizes for all reefs (a). This term is also used in parentage kernel fitting, but reef sizes are substituted as a proxy for pop size. This is should be bootstrapped to account for uncertainty.
prop_samp_vec <- as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)])#vector of proportion of habitat sampled for all reefs in time period t- I think that I should be coming up with a different value for this, but for now this will help me code the function

#bootstrap the particle AND parentage data and re-fit kernels and perform bode likelihood function for dispersal event matching
num_samples_sim <- nrow(SimConn) #because each row in SimConn is a particle the number of rows is N samples per iteration for bootstrapping

#make table to hold results
BootKernels <- data.table(data_source=character(), time_scale=character(), time_id=character(), k=numeric(), theta=numeric(),num_matches=numeric(), mdd=numeric(), med=numeric(), dist90=numeric(), log_like=numeric(), iteration=numeric())

BootLikelihood <- data.table(comparison=character(), time_scale=character(), data_source=character(), log_like=numeric(), iteration=numeric())

pb <- txtProgressBar(min = 0, max = 1000, style = 3)
AllRecruitsGen <- fread("~/parentage/kernel_fitting/1340_loci/final_results/tables/AllFishObsWithPar.csv")
MonsoonRecSampPar <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/RecruitsByMonsoon2012-14ForROMSComp.csv")


 AllRecruitsGenWithMonsoon <- left_join(AllRecruitsGen, MonsoonRecSampPar[, .(fish_indiv, year, monsoon=season)], by=c("fish_indiv", "year"))[
        offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"][year %in% c(2012, 2013, 2014)]    
 #    , .SD[sample(.N, 394, replace=TRUE)]] #resample the recruits with replacement not by the same number at each site

TotalOffsBySiteYear <- AllRecruitsGenWithMonsoon[, .(num_gen=.N), by=c("year", "site")]

AllRecruitsGenWithMonsoon2 <- left_join(AllRecruitsGenWithMonsoon, TotalOffsBySiteYear, by=c("year", "site"))

#######################################
##for kernels, use the matrices with NS magbangon seperated
#TotalOffsBySiteYear <- AllRecruitsGenWithMonsoon[, .(num_gen=.N), by=c("year", "site")]
#AllRecruitsGenWithMonsoon2 <- left_join(AllRecruitsGenWithMonsoon, TotalOffsBySiteYear, by=c("year", "site"))
#Centroids <- fread(file="~/oceanography/empirical_data/site_centroids_SimTest.csv") 
#Centroids$site <- gsub("_full", "", Centroids$site, fixed=TRUE)
#Centroids$site <- gsub("_ten_per_cover", "", Centroids$site, fixed=TRUE)
#
#Centroids <- Centroids %>%
#    arrange(site)
#### List of source locations
#SitesSource <- Centroids
#
#### List of destination locations
#SitesDest <- Centroids
#
#DistMatm <- distm(SitesSource[,c('lon','lat')], SitesSource[,c('lon','lat')], fun=distVincentyEllipsoid)
#Distances <- DistMatm*10^-3
#
#Area <- fread("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels_test.csv") %>%
#    arrange(site) %>%
#    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
#    mutate(kmsq=msq*10^-6)# %>%
#    #select(kmsq) #need to uncomment for functions to work
#Area$site <- gsub("_ten_per_cover", "", Area$site, fixed=TRUE)
#reef_sizes <- as.matrix(Area$kmsq)
#
#SiteIndex <- unique(Centroids %>% arrange(site), by="site")[, index := .I] #add the row number as the unique site index, leave CAI in if fitting a kernel SurveyData <- fread("~/oceanography/script_output/SurveyData/SurveyData.csv")
#######################################
for(i in 1:1000){
    
    #resample dispersal data, both genetics and simulation
    SimConnInt2 <- SimConn[, .SD[sample(.N, num_samples_sim, replace=TRUE)]] #sample the particle data
    #subsampling the particle data
    SimConnInt2SubSample <- SimConn[!is.na(dest_num_rec_sampled_annual), .SD[sample(.N, dest_num_rec_sampled_annual, replace=TRUE)], by=c("year", "dest")] #subsample the particle data according to the number of recruits we sampled at each site
    
    SimConnInt2SubSamplePar <- SimConnInt2SubSample[!is.na(dest_num_rec_sampled_annual)& source !="unknown", .SD[sample(.N, num_annual_parentage_matches, replace=TRUE)], by="year"] #randomly assign parentage based on the number of matches from that year and use the source prop_samp as the probability weighting
    SimConnInt2SubSample[particle_id %!in% SimConnInt2SubSamplePar[, particle_id], source := "unknown"]#and then for every particle that we didn't assign as a match, label the source unknown so it goes in the unassigned row
    #nrow(SimConnInt2SubSample[source=="unknown"])+nrow(SimConnInt2SubSample[source!="unknown"]) #should equal 394onnInt2SubSample <- SimConn[!is.na(dest_num_rec_sampled_annual), .SD[sample(.N, dest_num_rec_sampled_annual, replace=TRUE)], by=c("year", "dest")] 

#make one data table to rule them all....

 #   AllRecruitsGenWithMonsoon <- left_join(AllRecruitsGen, MonsoonRecSampPar[, .(fish_indiv, year, monsoon=season)], by=c("fish_indiv", "year"))[
 #       offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"][year %in% c(2012, 2013, 2014)]    
 ##    , .SD[sample(.N, 394, replace=TRUE)]] #resample the recruits with replacement not by the same number at each site
 #  
    #TotalOffsBySiteYear <- AllRecruitsGenWithMonsoon[, .(num_gen=.N), by=c("year", "site")]
    
    #test case redoing with subsampling by site and year instead of total recruits sampled
    AllRecruitsGenWithMonsoonInt <- AllRecruitsGenWithMonsoon2[, .SD[sample(.N, num_gen, replace=TRUE)], by=c("year", "site")] #resample the recruits with replacement
    #AllRecruitsGenWithMonsoonInt <- AllRecruitsGenWithMonsoon2[, .SD[sample(.N, num_gen, replace=TRUE)], by=c("year", "site")] #resample the recruits with replacement


    
        #format the matrices for kernel fitting and likelihood function
        genetic_parentage_matrices <- suppressMessages(format_genetic_parentage_matrix(AllRecruitsGenWithMonsoon=AllRecruitsGenWithMonsoonInt, SurveyData))
        genetic_kernel_matrices <- suppressMessages(format_genetic_kernel_parentage_matrix(AllRecruitsGenWithMonsoon=AllRecruitsGenWithMonsoonInt, SurveyData))
        
        biophys_norm_matrices <- format_biophys_normalized_matrix(SimConn=SimConnInt2, total_release_days=687, AddDestSim = AddDestSim)
        biophys_parentage_matrices <- format_biophys_parentage_matrix(SimConn=SimConnInt2, total_release_days=687, AddDestSim = AddDestSim)
        
        biophys_norm_matrices_subsample <- format_biophys_normalized_matrix(SimConn=SimConnInt2SubSample, total_release_days=687, AddDestSim = AddDestSim)
        biophys_parentage_matrices_subsample <- format_biophys_parentage_matrix(SimConn=SimConnInt2SubSample, total_release_days=687, AddDestSim = AddDestSim)



##fit the kernels
biophys_par_data2012 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2012), ncol=1, 1))
Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012, control=list(maxit=500)))

biophys_par_data2013 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2013), ncol=1, 1))
Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2013, control=list(maxit=500)))

biophys_par_data2014 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2014), ncol=1, 1))
Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2014, control=list(maxit=500)))

biophys_par_data2012_4 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$FullBiophysMat, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2014)))
Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012_4, control=list(maxit=500)))

biophys_par_dataNEM <- list(Distances=Distances, Assignments=biophys_parentage_matrices$MonsoonBiophysMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$MonsoonBiophysMatNEM), ncol=1, 1))
SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataNEM, control=list(maxit=500)))

biophys_par_dataSWM <- list(Distances=Distances, Assignments=biophys_parentage_matrices$MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$MonsoonBiophysMatSWM), ncol=1, 1))
SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataSWM, control=list(maxit=500)))

#subsampled simulation data
biophys_par_data2012_subsample <- list(Distances=Distances, Assignments=biophys_parentage_matrices_subsample$AnnualBiophysMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2012 & prop_anem_samp >0, .(prop_anem_samp)]))
Sim2012Fit_subsample <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012_subsample, control=list(maxit=500)))

biophys_par_data2013_subsample <- list(Distances=Distances, Assignments=biophys_parentage_matrices_subsample$AnnualBiophysMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2013 & prop_anem_samp >0, .(prop_anem_samp)]))
Sim2013Fit_subsample <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2013_subsample, control=list(maxit=500)))

biophys_par_data2014_subsample <- list(Distances=Distances, Assignments=biophys_parentage_matrices_subsample$AnnualBiophysMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Sim2014Fit_subsample <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2014_subsample, control=list(maxit=500)))

biophys_par_data2012_4_subsample <- list(Distances=Distances, Assignments=biophys_parentage_matrices_subsample$FullBiophysMat, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Sim2012_4Fit_subsample <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012_4_subsample, control=list(maxit=500)))

biophys_par_dataNEM_subsample <- list(Distances=Distances, Assignments=biophys_parentage_matrices_subsample$MonsoonBiophysMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
SimNEMFit_subsample <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataNEM_subsample, control=list(maxit=500)))

biophys_par_dataSWM_subsample <- list(Distances=Distances, Assignments=biophys_parentage_matrices_subsample$MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
SimSWMFit_subsample <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataSWM_subsample, control=list(maxit=500)))

#genetic data with same reef metadata structure as model

par_data2012 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2012 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012, control=list(maxit=500)))

par_data2013 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2013 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2013, control=list(maxit=500)))

par_data2014 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2014, control=list(maxit=500)))

par_data2012_4 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2012_4, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012_4, control=list(maxit=500)))

par_dataNEM <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014  & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014   & prop_anem_samp >0, .(prop_anem_samp)]))
GenNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataNEM, control=list(maxit=500)))

par_dataSWM <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014  & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014   & prop_anem_samp >0, .(prop_anem_samp)]))
GenSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataSWM, control=list(maxit=500)))


#make a summary table for all time frames of simulation data
SimKernelsInt <- as.data.frame(matrix(NA, nrow = 12, ncol = 10))
setDT(SimKernelsInt)
setnames(SimKernelsInt, c("data_source", "time_scale", "time_id", "k", "theta","num_matches", "mdd", "med", "dist90", "log_like"))


BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[1] <- i
SimKernelsInt$data_source[1] <- "simulation"
SimKernelsInt$time_scale[1] <- "annual"
SimKernelsInt$time_id[1] <- "2012"
SimKernelsInt$k[1] <-  BestK2012
SimKernelsInt$theta[1] <- BestTheta2012
SimKernelsInt$num_matches[1] <-sum(biophys_par_data2012$Assignments[1:nrow(biophys_par_data2012$Assignments)-1,])
SimKernelsInt$mdd[1] <- MDD2012
SimKernelsInt$med[1] <- Med2012
SimKernelsInt$dist90[1] <- Dist90_2012
SimKernelsInt$log_like[1] <- logLik(Sim2012Fit)[1]

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[2] <- i
SimKernelsInt$data_source[2] <- "simulation"
SimKernelsInt$time_scale[2] <- "annual"
SimKernelsInt$time_id[2] <- "2013"
SimKernelsInt$k[2] <-  BestK2013
SimKernelsInt$theta[2] <- BestTheta2013
SimKernelsInt$num_matches[2] <-sum(biophys_par_data2013$Assignments[1:nrow(biophys_par_data2013$Assignments)-1,])
SimKernelsInt$mdd[2] <- MDD2013
SimKernelsInt$med[2] <- Med2013
SimKernelsInt$dist90[2] <- Dist90_2013
SimKernelsInt$log_like[2] <- logLik(Sim2013Fit)[1]


BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[3] <- i
SimKernelsInt$data_source[3] <- "simulation"
SimKernelsInt$time_scale[3] <- "annual"
SimKernelsInt$time_id[3] <- "2014"
SimKernelsInt$k[3] <-  BestK2014
SimKernelsInt$theta[3] <- BestTheta2014
SimKernelsInt$num_matches[3] <-sum(biophys_par_data2014$Assignments[1:nrow(biophys_par_data2014$Assignments)-1,])
SimKernelsInt$mdd[3] <- MDD2014
SimKernelsInt$med[3] <- Med2014
SimKernelsInt$dist90[3] <- Dist90_2014
SimKernelsInt$log_like[3] <- logLik(Sim2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Sim2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)
    
SimKernelsInt$iteration[4] <- i
SimKernelsInt$data_source[4] <- "simulation"
SimKernelsInt$time_scale[4] <- "total_years"
SimKernelsInt$time_id[4] <- "2012-4"
SimKernelsInt$k[4] <-  BestK2012_4
SimKernelsInt$theta[4] <- BestTheta2012_4
SimKernelsInt$num_matches[4] <-sum(biophys_par_data2012_4$Assignments[1:nrow(biophys_par_data2012_4$Assignments)-1,])
SimKernelsInt$mdd[4] <- MDD2012_4
SimKernelsInt$med[4] <- Med2012_4
SimKernelsInt$dist90[4] <- Dist90_2012_4
SimKernelsInt$log_like[4] <- logLik(Sim2012_4Fit)[1]

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[5] <- i
SimKernelsInt$data_source[5] <- "simulation"
SimKernelsInt$time_scale[5] <- "monsoonal"
SimKernelsInt$time_id[5] <- "NEM"
SimKernelsInt$k[5] <-  BestKNEM
SimKernelsInt$theta[5] <- BestThetaNEM
SimKernelsInt$num_matches[5] <-sum(biophys_par_dataNEM$Assignments[1:nrow(biophys_par_dataNEM$Assignments)-1,])
SimKernelsInt$mdd[5] <- MDDNEM
SimKernelsInt$med[5] <- MedNEM
SimKernelsInt$dist90[5] <- Dist90_NEM
SimKernelsInt$log_like[5] <- logLik(SimNEMFit)[1]

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[6] <- i
SimKernelsInt$data_source[6] <- "simulation"
SimKernelsInt$time_scale[6] <- "monsoonal"
SimKernelsInt$time_id[6] <- "SWM"
SimKernelsInt$k[6] <-  BestKSWM
SimKernelsInt$theta[6] <- BestThetaSWM
SimKernelsInt$num_matches[6] <-sum(biophys_par_dataSWM$Assignments[1:nrow(biophys_par_dataSWM$Assignments)-1,])
SimKernelsInt$mdd[6] <- MDDSWM
SimKernelsInt$med[6] <- MedSWM
SimKernelsInt$dist90[6] <- Dist90_SWM
SimKernelsInt$log_like[6] <- logLik(SimSWMFit)[1]

#add the subsampled results
    
BestK2012 <- as.numeric(coef(Sim2012Fit_subsample)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit_subsample)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[7] <- i
SimKernelsInt$data_source[7] <- "simulation_subsample"
SimKernelsInt$time_scale[7] <- "annual"
SimKernelsInt$time_id[7] <- "2012"
SimKernelsInt$k[7] <-  BestK2012
SimKernelsInt$theta[7] <- BestTheta2012
SimKernelsInt$num_matches[7] <-sum(biophys_par_data2012_subsample$Assignments[1:nrow(biophys_par_data2012_subsample$Assignments)-1,])
SimKernelsInt$mdd[7] <- MDD2012
SimKernelsInt$med[7] <- Med2012
SimKernelsInt$dist90[7] <- Dist90_2012
SimKernelsInt$log_like[7] <- logLik(Sim2012Fit_subsample)[1]

BestK2013 <- as.numeric(coef(Sim2013Fit_subsample)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit_subsample)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[8] <- i
SimKernelsInt$data_source[8] <- "simulation_subsample"
SimKernelsInt$time_scale[8] <- "annual"
SimKernelsInt$time_id[8] <- "2013"
SimKernelsInt$k[8] <-  BestK2013
SimKernelsInt$theta[8] <- BestTheta2013
SimKernelsInt$num_matches[8] <-sum(biophys_par_data2013_subsample$Assignments[1:nrow(biophys_par_data2013_subsample$Assignments)-1,])
SimKernelsInt$mdd[8] <- MDD2013
SimKernelsInt$med[8] <- Med2013
SimKernelsInt$dist90[8] <- Dist90_2013
SimKernelsInt$log_like[8] <- logLik(Sim2013Fit_subsample)[1]


BestK2014 <- as.numeric(coef(Sim2014Fit_subsample)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit_subsample)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[9] <- i
SimKernelsInt$data_source[9] <- "simulation_subsample"
SimKernelsInt$time_scale[9] <- "annual"
SimKernelsInt$time_id[9] <- "2014"
SimKernelsInt$k[9] <-  BestK2014
SimKernelsInt$theta[9] <- BestTheta2014
SimKernelsInt$num_matches[9] <-sum(biophys_par_data2014_subsample$Assignments[1:nrow(biophys_par_data2014_subsample$Assignments)-1,])
SimKernelsInt$mdd[9] <- MDD2014
SimKernelsInt$med[9] <- Med2014
SimKernelsInt$dist90[9] <- Dist90_2014
SimKernelsInt$log_like[9] <- logLik(Sim2014Fit_subsample)[1]

BestK2012_4 <- as.numeric(coef(Sim2012_4Fit_subsample)[1])
BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit_subsample)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)
    
SimKernelsInt$iteration[10] <- i
SimKernelsInt$data_source[10] <- "simulation_subsample"
SimKernelsInt$time_scale[10] <- "total_years"
SimKernelsInt$time_id[10] <- "2012-4"
SimKernelsInt$k[10] <-  BestK2012_4
SimKernelsInt$theta[10] <- BestTheta2012_4
SimKernelsInt$num_matches[10] <-sum(biophys_par_data2012_4_subsample$Assignments[1:nrow(biophys_par_data2012_4_subsample$Assignments)-1,])
SimKernelsInt$mdd[10] <- MDD2012_4
SimKernelsInt$med[10] <- Med2012_4
SimKernelsInt$dist90[10] <- Dist90_2012_4
SimKernelsInt$log_like[10] <- logLik(Sim2012_4Fit_subsample)[1]

BestKNEM <- as.numeric(coef(SimNEMFit_subsample)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit_subsample)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[11] <- i
SimKernelsInt$data_source[11] <- "simulation_subsample"
SimKernelsInt$time_scale[11] <- "monsoonal"
SimKernelsInt$time_id[11] <- "NEM"
SimKernelsInt$k[11] <-  BestKNEM
SimKernelsInt$theta[11] <- BestThetaNEM
SimKernelsInt$num_matches[11] <-sum(biophys_par_dataNEM_subsample$Assignments[1:nrow(biophys_par_dataNEM_subsample$Assignments)-1,])
SimKernelsInt$mdd[11] <- MDDNEM
SimKernelsInt$med[11] <- MedNEM
SimKernelsInt$dist90[11] <- Dist90_NEM
SimKernelsInt$log_like[11] <- logLik(SimNEMFit_subsample)[1]

BestKSWM <- as.numeric(coef(SimSWMFit_subsample)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit_subsample)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[12] <- i
SimKernelsInt$data_source[12] <- "simulation_subsample"
SimKernelsInt$time_scale[12] <- "monsoonal"
SimKernelsInt$time_id[12] <- "SWM"
SimKernelsInt$k[12] <-  BestKSWM
SimKernelsInt$theta[12] <- BestThetaSWM
SimKernelsInt$num_matches[12] <-sum(biophys_par_dataSWM_subsample$Assignments[1:nrow(biophys_par_dataSWM_subsample$Assignments)-1,])
SimKernelsInt$mdd[12] <- MDDSWM
SimKernelsInt$med[12] <- MedSWM
SimKernelsInt$dist90[12] <- Dist90_SWM
SimKernelsInt$log_like[12] <- logLik(SimSWMFit_subsample)[1]

    
#make a summary table for all time frames of genetic data
GenKernelsInt <- as.data.frame(matrix(NA, nrow = 6, ncol = 10))
setDT(GenKernelsInt)
setnames(GenKernelsInt, c("data_source", "time_scale", "time_id", "k", "theta", "num_matches", "mdd", "med", "dist90", "log_like"))
#GenKernelsInt <- data.table(data_source=time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric())


BestK2012 <- as.numeric(coef(Gen2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Gen2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[1] <- i
GenKernelsInt$data_source[1] <- "genetics"    
GenKernelsInt$time_scale[1] <- "annual"
GenKernelsInt$time_id[1] <- "2012"
GenKernelsInt$k[1] <-  BestK2012
GenKernelsInt$theta[1] <- BestTheta2012
GenKernelsInt$num_matches[1] <-sum(par_data2012$Assignments[1:nrow(par_data2012$Assignments)-1,])
GenKernelsInt$mdd[1] <- MDD2012
GenKernelsInt$med[1] <- Med2012
GenKernelsInt$dist90[1] <- Dist90_2012
GenKernelsInt$log_like[1] <- logLik(Gen2012Fit)[1]

BestK2013 <- as.numeric(coef(Gen2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Gen2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[2] <- i
GenKernelsInt$data_source[2] <- "genetics"    
GenKernelsInt$time_scale[2] <- "annual"
GenKernelsInt$time_id[2] <- "2013"
GenKernelsInt$k[2] <-  BestK2013
GenKernelsInt$theta[2] <- BestTheta2013
GenKernelsInt$num_matches[2] <-sum(par_data2013$Assignments[1:nrow(par_data2013$Assignments)-1,])
GenKernelsInt$mdd[2] <- MDD2013
GenKernelsInt$med[2] <- Med2013
GenKernelsInt$dist90[2] <- Dist90_2013
GenKernelsInt$log_like[2] <- logLik(Gen2013Fit)[1]

BestK2014 <- as.numeric(coef(Gen2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Gen2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[3] <- i
GenKernelsInt$data_source[3] <- "genetics"    
GenKernelsInt$time_scale[3] <- "annual"
GenKernelsInt$time_id[3] <- "2014"
GenKernelsInt$k[3] <-  BestK2014
GenKernelsInt$theta[3] <- BestTheta2014
GenKernelsInt$num_matches[3] <-sum(par_data2014$Assignments[1:nrow(par_data2014$Assignments)-1,])
GenKernelsInt$mdd[3] <- MDD2014
GenKernelsInt$med[3] <- Med2014
GenKernelsInt$dist90[3] <- Dist90_2014
GenKernelsInt$log_like[3] <- logLik(Gen2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Gen2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Gen2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[4] <- i
GenKernelsInt$data_source[4] <- "genetics"    
GenKernelsInt$time_scale[4] <- "total_years"
GenKernelsInt$time_id[4] <- "2012-4"
GenKernelsInt$k[4] <-  BestK2012_4
GenKernelsInt$theta[4] <- BestTheta2012_4
GenKernelsInt$num_matches[4] <-sum(par_data2012_4$Assignments[1:nrow(par_data2012_4$Assignments)-1,])
GenKernelsInt$mdd[4] <- MDD2012_4
GenKernelsInt$med[4] <- Med2012_4
GenKernelsInt$dist90[4] <- Dist90_2012_4
GenKernelsInt$log_like[4] <- logLik(Gen2012_4Fit)[1]

BestKNEM <- as.numeric(coef(GenNEMFit)[1])
BestThetaNEM <- as.numeric(coef(GenNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[5] <- i
GenKernelsInt$data_source[5] <- "genetics"    
GenKernelsInt$time_scale[5] <- "monsoonal"
GenKernelsInt$time_id[5] <- "NEM"
GenKernelsInt$k[5] <-  BestKNEM
GenKernelsInt$theta[5] <- BestThetaNEM
GenKernelsInt$num_matches[5] <-sum(par_dataNEM$Assignments[1:nrow(par_dataNEM$Assignments)-1,])
GenKernelsInt$mdd[5] <- MDDNEM
GenKernelsInt$med[5] <- MedNEM
GenKernelsInt$dist90[5] <- Dist90_NEM
GenKernelsInt$log_like[5] <- logLik(GenNEMFit)[1]

BestKSWM <- as.numeric(coef(GenSWMFit)[1])
BestThetaSWM <- as.numeric(coef(GenSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[6] <- i
GenKernelsInt$data_source[6] <- "genetics"    
GenKernelsInt$time_scale[6] <- "monsoonal"
GenKernelsInt$time_id[6] <- "SWM"
GenKernelsInt$k[6] <-  BestKSWM
GenKernelsInt$theta[6] <- BestThetaSWM
GenKernelsInt$num_matches[6] <-sum(par_dataSWM$Assignments[1:nrow(par_dataSWM$Assignments)-1,])
GenKernelsInt$mdd[6] <- MDDSWM
GenKernelsInt$med[6] <- MedSWM
GenKernelsInt$dist90[6] <- Dist90_SWM
GenKernelsInt$log_like[6] <- logLik(GenSWMFit)[1]

GenKernelsInt$iteration <- i
GenKernelsInt$data_source <- "genetics"

        BootKernelsInt <- rbind(SimKernelsInt, GenKernelsInt)
   
        BootKernels <- rbind(BootKernels, BootKernelsInt)
    
 #also bootstrap the biophysical likelihood
Data_par2012_bio2012 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2012[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012[1:nrow(genetic_parentage_matrices$GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2012,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012[nrow(genetic_parentage_matrices$GenMat2012),]))

Data_par2013_bio2013 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2013[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2013[1:nrow(genetic_parentage_matrices$GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2013,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2013[nrow(genetic_parentage_matrices$GenMat2013),]))

Data_par2014_bio2014 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2014[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2014[1:nrow(genetic_parentage_matrices$GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2014[nrow(genetic_parentage_matrices$GenMat2014),]))


#total compared to total
Data_par2012_4_bio2012_4 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012_4[1:nrow(genetic_parentage_matrices$GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012_4[nrow(genetic_parentage_matrices$GenMat2012_4),]))

#for seasonal kernels

 Data_parNEM_bioNEM <- list(BioPhysMat=as.matrix(biophys_norm_matrices$MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(genetic_parentage_matrices$GenMatNEM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),]))

 Data_parSWM_bioSWM <- list(BioPhysMat=as.matrix(biophys_norm_matrices$MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(genetic_parentage_matrices$GenMatSWM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))
#total to total 
 Data_parNEMSWM_bio2012_4 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=(as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(genetic_parentage_matrices$GenMatNEM)-1,])+
                                  as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(genetic_parentage_matrices$GenMatSWM)-1,])), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),])+
                    as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))

#calculate likelihood
LL_annual_annual <- LL_biophys(Data_par2012_bio2012)+LL_biophys(Data_par2013_bio2013)+LL_biophys(Data_par2014_bio2014)
LL_total_total <- LL_biophys(Data_par2012_4_bio2012_4)
LL_monsoonal_monsoonal <- LL_biophys(Data_parNEM_bioNEM)+LL_biophys(Data_parSWM_bioSWM)
LL_monsoonal_total <- LL_biophys(Data_parNEMSWM_bio2012_4)

#subsampled simulation data
Data_par2012_bio2012_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$AnnualBiophysMatNorm2012[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012[1:nrow(genetic_parentage_matrices$GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2012,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012[nrow(genetic_parentage_matrices$GenMat2012),]))

Data_par2013_bio2013_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$AnnualBiophysMatNorm2013[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2013[1:nrow(genetic_parentage_matrices$GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2013,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2013[nrow(genetic_parentage_matrices$GenMat2013),]))

Data_par2014_bio2014_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$AnnualBiophysMatNorm2014[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2014[1:nrow(genetic_parentage_matrices$GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2014[nrow(genetic_parentage_matrices$GenMat2014),]))


#total compared to total
Data_par2012_4_bio2012_4_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012_4[1:nrow(genetic_parentage_matrices$GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012_4[nrow(genetic_parentage_matrices$GenMat2012_4),]))

#for seasonal kernels

 Data_parNEM_bioNEM_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(genetic_parentage_matrices$GenMatNEM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),]))

 Data_parSWM_bioSWM_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(genetic_parentage_matrices$GenMatSWM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))
#total to total 
 Data_parNEMSWM_bio2012_4_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=(as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(genetic_parentage_matrices$GenMatNEM)-1,])+
                                  as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(genetic_parentage_matrices$GenMatSWM)-1,])), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),])+
                    as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))

#calculate likelihood
LL_annual_annual_subsample <- LL_biophys(Data_par2012_bio2012_subsample)+LL_biophys(Data_par2013_bio2013_subsample)+LL_biophys(Data_par2014_bio2014_subsample)
LL_total_total_subsample <- LL_biophys(Data_par2012_4_bio2012_4_subsample)
LL_monsoonal_monsoonal_subsample <- LL_biophys(Data_parNEM_bioNEM_subsample)+LL_biophys(Data_parSWM_bioSWM_subsample)
LL_monsoonal_total_subsample <- LL_biophys(Data_parNEMSWM_bio2012_4_subsample)
    
#store results
LLInt <- data.table(comparison= c("gen_total_bio_total", "gen_annual_bio_annual", "gen_monsoonal_bio_monsoonal", "gen_monsoonal_bio_total", "gen_total_bio_total", "gen_annual_bio_annual", "gen_monsoonal_bio_monsoonal", "gen_monsoonal_bio_total"), 
           time_scale= c("total_years", "annual", "total_monsoons", "monsoonal"),
           log_like=c(LL_total_total, LL_annual_annual, LL_monsoonal_monsoonal, LL_monsoonal_total, LL_total_total_subsample, LL_annual_annual_subsample, LL_monsoonal_monsoonal_subsample, LL_monsoonal_total_subsample), 
            data_source = c("simulation","simulation","simulation","simulation", "simulation_subsample", "simulation_subsample","simulation_subsample","simulation_subsample"))
LLInt$iteration <- i
    
BootLikelihood <- rbind(BootLikelihood, LLInt)
    
setTxtProgressBar(pb, i)
    }
close(pb)

#fwrite(BootKernels, "~/oceanography/script_output/KernelFits/BootstrappedBiophysicalKernels08DayPLD.csv")
#fwrite(BootLikelihood, "~/oceanography/script_output/KernelFits/BootstrappedBiophysicalLikelihoods08DayPLD.csv")

  |======                                                                |   9%

In [42]:
SimConnInt2SubSample

year,dest,particle_id,source,monsoon,date,num_annual_parentage_matches,num_release_days_seasonal,num_release_days_annual,dest_num_females,...,source_num_rec_sampled_annual,source_prop_anem_samp,sim_month,sim_day,sim_year,daily_particles_rec,daily_particles_released,dist_km,bearing,direction
2012,Palanas,P57062220,unknown,SWM,2011-10-14,3,31,244,49,...,NA,NA,10,10,2011,1849,2878545920,NA,NA,NA
2012,Palanas,P83526828,unknown,NEM,2012-02-18,3,213,244,49,...,NA,NA,2,2,2012,2116,4619166656,NA,NA,NA
2012,Palanas,P94591376,unknown,NEM,2012-04-01,3,213,244,49,...,NA,NA,4,4,2012,24025,1934023040,NA,NA,NA
2012,Palanas,P83526842,unknown,NEM,2012-02-18,3,213,244,49,...,NA,NA,2,2,2012,2116,4619166656,NA,NA,NA
2012,Palanas,P82827306,unknown,NEM,2012-02-15,3,213,244,49,...,NA,NA,2,2,2012,10816,5334305408,NA,NA,NA
2012,Palanas,P59478347,unknown,SWM,2011-10-25,3,31,244,49,...,NA,NA,10,10,2011,576,3472246016,NA,NA,NA
2012,Palanas,P80620801,unknown,NEM,2012-02-06,3,213,244,49,...,NA,NA,2,2,2012,5329,5091428096,NA,NA,NA
2012,Palanas,P76627116,unknown,NEM,2012-01-17,3,213,244,49,...,NA,NA,1,1,2012,15625,9688106112,NA,NA,NA
2012,Palanas,P62625948,unknown,NEM,2011-11-10,3,213,244,49,...,NA,NA,11,11,2011,2916,5536703168,NA,NA,NA
2012,Palanas,P79763153,unknown,NEM,2012-02-01,3,213,244,49,...,NA,NA,2,2,2012,484,3652155136,NA,NA,NA


In [13]:
biophys_par_data2012_subsample

0.0000000,36.59466,3.0328732,3.7812405,39.544910,35.144215,36.419898,40.865998,7.7170642,15.862935,...,2.6540921,12.7521560,11.9430714,11.3862533,24.7874862,10.8525552,6.9584971,23.276705,13.915290,2.1280885
36.5946593,0.00000,37.5112742,37.6253510,32.701393,28.926277,27.351389,33.695741,38.1057602,39.049765,...,36.2377630,38.6352916,38.8423125,38.7832324,37.2650016,38.8627857,38.1614224,37.940511,38.551899,36.4035501
3.0328732,37.51127,0.0000000,0.7611792,37.871146,33.627822,35.066563,39.166201,4.7583636,13.024371,...,5.6803278,9.8703541,9.0230795,8.4572682,22.3207572,7.9035693,3.9724282,20.716564,11.069808,5.1471170
3.7812405,37.62535,0.7611792,0.0000000,37.352525,33.148293,34.623834,38.640953,3.9999295,12.278269,...,6.4329063,9.1192352,8.2669730,7.7002819,21.6392752,7.1445494,3.2116192,20.017227,10.323743,5.9014240
39.5449097,32.70139,37.8711462,37.3525247,0.000000,4.863776,5.420621,1.370390,34.4925112,28.304747,...,41.3312751,30.6573228,31.5336407,31.9607107,18.7098394,32.4885497,35.1757138,20.679329,29.579123,41.0485538
35.1442145,28.92628,33.6278217,33.1482933,4.863776,0.000000,2.533995,6.223798,30.5139243,24.963682,...,36.8276566,27.0362084,27.8664339,28.2505730,16.1946825,28.7460208,31.1571402,18.037808,26.040679,36.5673271
36.4198977,27.35139,35.0665635,34.6238340,5.420621,2.533995,0.000000,6.580623,32.1779180,27.009579,...,37.9754549,28.9363496,29.7373991,30.0955838,18.5624726,30.5699476,32.7894955,20.362496,27.988733,37.7441889
40.8659979,33.69574,39.1662013,38.6409535,1.370390,6.223798,6.580623,0.000000,35.7417602,29.438628,...,42.6682007,31.8425287,32.7266049,33.1614679,19.7145272,33.6949350,36.4318492,21.697643,30.749828,42.3820949
7.7170642,38.10576,4.7583636,3.9999295,34.492511,30.513924,32.177918,35.741760,0.0000000,8.315917,...,10.3654013,5.1402209,4.2700861,3.7016111,17.9802829,3.1454389,0.8152421,16.270087,6.366148,9.8446575
15.8629348,39.04977,13.0243709,12.2782690,28.304747,24.963682,27.009579,29.438628,8.3159171,0.000000,...,18.4582391,3.1819446,4.1010325,4.6708512,10.2978939,5.2677875,9.1296641,8.395073,1.954587,17.9621025
24.4585460,37.82302,21.9388356,21.2468135,19.483034,16.994224,19.361938,20.477052,17.5336836,9.713904,...,26.8648033,12.6833762,13.6344936,14.1717075,0.7997638,14.7679554,18.3347502,1.322755,11.440425,26.4270338


In [ ]:
#subsampled simulation data
Data_par2012_bio2012_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$AnnualBiophysMatNorm2012[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012[1:nrow(genetic_parentage_matrices$GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2012,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012[nrow(genetic_parentage_matrices$GenMat2012),]))

Data_par2013_bio2013_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$AnnualBiophysMatNorm2013[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2013[1:nrow(genetic_parentage_matrices$GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2013,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2013[nrow(genetic_parentage_matrices$GenMat2013),]))

Data_par2014_bio2014_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$AnnualBiophysMatNorm2014[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2014[1:nrow(genetic_parentage_matrices$GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2014[nrow(genetic_parentage_matrices$GenMat2014),]))


#total compared to total
Data_par2012_4_bio2012_4_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012_4[1:nrow(genetic_parentage_matrices$GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012_4[nrow(genetic_parentage_matrices$GenMat2012_4),]))

#for seasonal kernels

 Data_parNEM_bioNEM_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(genetic_parentage_matrices$GenMatNEM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),]))

 Data_parSWM_bioSWM_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(genetic_parentage_matrices$GenMatSWM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))
#total to total 
 Data_parNEMSWM_bio2012_4_subsample <- list(BioPhysMat=as.matrix(biophys_norm_matrices_subsample$FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=(as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(genetic_parentage_matrices$GenMatNEM)-1,])+
                                  as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(genetic_parentage_matrices$GenMatSWM)-1,])), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),])+
                    as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))

#calculate likelihood
LL_annual_annual_subsample <- LL_biophys(Data_par2012_bio2012_subsample)+LL_biophys(Data_par2013_bio2013_subsample)+LL_biophys(Data_par2014_bio2014_subsample)
LL_total_total_subsample <- LL_biophys(Data_par2012_4_bio2012_4_subsample)
LL_monsoonal_monsoonal_subsample <- LL_biophys(Data_parNEM_bioNEM_subsample)+LL_biophys(Data_parSWM_bioSWM_subsample)
LL_monsoonal_total_subsample <- LL_biophys(Data_parNEMSWM_bio2012_4_subsample)

In [ ]:
LL_annual_annual_subsample

In [ ]:
par_data2014 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))


In [ ]:
par_data2012 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2012 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012, control=list(maxit=500)))

par_data2013 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2013 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2013, control=list(maxit=500)))

par_data2014 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2014, control=list(maxit=500)))

par_data2012_4 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2012_4, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012_4, control=list(maxit=500)))

par_dataNEM <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2014  & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014   & prop_anem_samp >0, .(prop_anem_samp)]))
GenNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataNEM, control=list(maxit=500)))

par_dataSWM <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2014  & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014   & prop_anem_samp >0, .(prop_anem_samp)]))
GenSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataSWM, control=list(maxit=500)))

